In [2]:
ls ../../

README.md           db.sqlite3          elevate/            src/
crm/                db.sqlite3-journal  manage.py*


# Connect to postgres database

In [ ]:
import psycopg2

DB_CONFIG = {
    'dbname': 'elevate_db',
    'user': 'eval_user',
    'password': 'eval_password',    
    'host': 'localhost',
    'port': 5432  # default PostgreSQL port
}
try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # Query all records from crm_membership table
    cursor.execute('SELECT * FROM crm_membership LIMIT 10')
    results = cursor.fetchall()

    # Get column names
    column_names = [desc[0] for desc in cursor.description]
    print(f"Columns: {column_names}")

    # Show number of records and sample data
    print(f"Number of records found: {len(results)}")
    if results:
        print("\nSample data (first 5 records):")
        for row in results[:5]:
            print(row)
    else:
        print("\nNo records found in the crm_membership table.")

except psycopg2.Error as e:
    print("Database error:", e)

finally:
    # Close connection
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()


Columns: ['id', 'name', 'membership_plan', 'membership_active', 'unique_code', 'created_at', 'updated_at']
Number of records found: 1

Sample data (first 5 records):
(1, 'Ali', 'P', True, '1000', datetime.datetime(2025, 6, 16, 21, 45, 35, 270525, tzinfo=datetime.timezone.utc), datetime.datetime(2025, 6, 16, 21, 45, 35, 270525, tzinfo=datetime.timezone.utc))


In [3]:
!pip install Faker


In [15]:
from faker import Faker
import psycopg2
import random
from datetime import datetime, timezone
import uuid

# Initialize Faker
fake = Faker()


# Generate a single fake membership row
def generate_fake_membership():
    name = fake.first_name()
    membership_plan = random.choice(['B', 'S', 'P'])  # Basic, Standard, Premium (adjust if needed)
    membership_active = random.choice([True, False])
    unique_code = str(random.randint(1000, 9999))
    now = datetime.now(timezone.utc)
    return (name, membership_plan, membership_active, unique_code, now, now)

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()

    insert_query = """
        INSERT INTO crm_membership
        (name, membership_plan, membership_active, unique_code, created_at, updated_at)
        VALUES (%s, %s, %s, %s, %s, %s)
    """

    # Generate 1000 fake rows
    fake_data = [generate_fake_membership() for _ in range(1000)]
    cursor.executemany(insert_query, fake_data)
    conn.commit()

    print("✅ Successfully inserted 1000 fake crm_membership records.")

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()


✅ Successfully inserted 1000 fake crm_membership records.


In [18]:
import psycopg2


try:
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # Fetch column names from the education_course table
    cursor.execute("""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_name = 'education_course'
        ORDER BY ordinal_position
    """)
    
    columns = [row[0] for row in cursor.fetchall()]
    print("✅ Columns in 'education_course':")
    for col in columns:
        print(f"- {col}")

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()


✅ Columns in 'education_course':
- id
- Course_title
- Course_description
- Course_start_date
- Course_end_date
- Course_instructor
- slug


In [24]:
import psycopg2
from faker import Faker
from datetime import timedelta, datetime, timezone
import random
from slugify import slugify  # pip install python-slugify

# Setup Faker
fake = Faker()


# Connect to PostgreSQL
conn = psycopg2.connect(**DB_CONFIG)
cursor = conn.cursor()

# Insert 100 fake courses
for _ in range(100):
    course_title = fake.sentence(nb_words=4).rstrip('.')
    course_description = fake.paragraph(nb_sentences=5)
    start_date = fake.date_between(start_date='-1y', end_date='today')
    end_date = start_date + timedelta(days=random.randint(30, 180))
    instructor = fake.name()
    slug = slugify(course_title)

    # Insert into Course table
# Insert into Course table
    cursor.execute("""
        INSERT INTO education_course 
            ("Course_title", "Course_description", "Course_start_date", "Course_end_date", "Course_instructor", "slug")
        VALUES (%s, %s, %s, %s, %s, %s)
        RETURNING id
    """, (course_title, course_description, start_date, end_date, instructor, slug))

    course_id = cursor.fetchone()[0]

    # Insert 5–10 lessons for the course
    for i in range(1, random.randint(5, 10) + 1):
        lesson_name = fake.sentence(nb_words=3).rstrip('.')
        lesson_content = fake.paragraph(nb_sentences=10)
        lesson_slug = slugify(lesson_name)

        cursor.execute("""
            INSERT INTO education_lesson (lesson_name, lesson_content, lesson_course_id, lesson_order, slug)
            VALUES (%s, %s, %s, %s, %s)
        """, (lesson_name, lesson_content, course_id, i, lesson_slug))

# Commit all changes
conn.commit()

# Close connections
cursor.close()
conn.close()

print("✅ 100 fake courses with lessons inserted into PostgreSQL.")


✅ 100 fake courses with lessons inserted into PostgreSQL.


In [8]:
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--Requirement already satisfied: packaging in /Users/parsanemati/miniconda3/envs/django/lib/python3.10/site-packages (from plotly) (25.0)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 33.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [plotly]2m1/2 [plotly]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [plotly]


In [11]:
import plotly.express as px
import numpy as np

# Generate some example housing prices
np.random.seed(42)
housing_prices = np.random.normal(loc=300_000, scale=50_000, size=500)

# Create histogram
fig = px.histogram(
    x=housing_prices,
    nbins=30,
    title="Distribution of Housing Prices",
    labels={"x": "Price (USD)", "y": "Count"},
)

# Customize appearance
fig.update_layout(
    bargap=0.1,
    template="plotly_white",
)

# Show the chart
fig.show()
